In [13]:
import numpy as np

# data I/O
data = open('rnn_input.txt', 'r').read() # should be simple plain text file
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print('data has %d characters, %d unique.' % (data_size, vocab_size))
char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }

# hyperparameters
hidden_size = 100 # size of hidden layer of neurons
seq_length = 25 # number of steps to unroll the RNN for
learning_rate = 1e-1

# model parameters
Wxh = np.random.randn(hidden_size, vocab_size)*0.01 # input to hidden 100 x 21
Whh = np.random.randn(hidden_size, hidden_size)*0.01 # hidden to hidden 100 x 100
Why = np.random.randn(vocab_size, hidden_size)*0.01 # hidden to output 21 x 100
bh = np.zeros((hidden_size, 1)) # hidden bias 100 x 1
by = np.zeros((vocab_size, 1)) # output bias 21 x 1



data has 47 characters, 21 unique.


In [14]:
def lossFun(inputs, targets, hprev):
  """
  inputs,targets are both list of integers.
  hprev is Hx1 array of initial hidden state
  returns the loss, gradients on model parameters, and last hidden state
  """
  xs, hs, ys, ps = {}, {}, {}, {}
  hs[-1] = np.copy(hprev)
  loss = 0
  # forward pass
  for t in range(len(inputs)):
    xs[t] = np.zeros((vocab_size,1)) # encode in 1-of-k representation 21 x 1 dict with 100 elements of 21 x 1
    xs[t][inputs[t]] = 1 # one particular column we update to 1
    hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t-1]) + bh) # hidden state 100 x 21 x 21 x 1 - 100 x 1
    ys[t] = np.dot(Why, hs[t]) + by # unnormalized log probabilities for next chars - 21 x 100 x 100 x 1 - 21 x 1
    ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) # probabilities for next chars 21 x 1
    loss += -np.log(ps[t][targets[t],0]) # softmax (cross-entropy loss)
  # backward pass: compute gradients going backwards
  dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why) # 100 x 21, 100 x 100, 21 x 100
  dbh, dby = np.zeros_like(bh), np.zeros_like(by) # 100 x 1, 21 x 1
  dhnext = np.zeros_like(hs[0]) # 100 x 1
  for t in reversed(range(len(inputs))):
    dy = np.copy(ps[t]) # 21 x 1
    dy[targets[t]] -= 1 # backprop into y. see http://cs231n.github.io/neural-networks-case-study/#grad if confused here
    dWhy += np.dot(dy, hs[t].T) # 21 x 1 x 1 x 100 - 21 x 100
    dby += dy # 21 x 1
    dh = np.dot(Why.T, dy) + dhnext # backprop into h  100 x 21 x 21 x 1 100 x 1
    dhraw = (1 - hs[t] * hs[t]) * dh # backprop through tanh nonlinearity 100 x 1
    dbh += dhraw # 100 x 1
    dWxh += np.dot(dhraw, xs[t].T) # 100 x 1 x 1 x 21 - 100 x 21
    dWhh += np.dot(dhraw, hs[t-1].T) # 100 x 1 x 1 x 100 - 100 x 100
    dhnext = np.dot(Whh.T, dhraw) # 100 x 100 x 100 x 1 - 100 x 1
  for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
    np.clip(dparam, -5, 5, out=dparam) # clip to mitigate exploding gradients
  return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]

In [15]:
def sample(h, seed_ix, n):
  """ 
  sample a sequence of integers from the model 
  h is memory state, seed_ix is seed letter for first time step
  """
  x = np.zeros((vocab_size, 1))
  x[seed_ix] = 1
  ixes = []
  for t in range(n):
    h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
    y = np.dot(Why, h) + by
    p = np.exp(y) / np.sum(np.exp(y))
    ix = np.random.choice(range(vocab_size), p=p.ravel())
    x = np.zeros((vocab_size, 1))
    x[ix] = 1
    ixes.append(ix)
  return ixes


In [18]:
n, p = 0, 0
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
mbh, mby = np.zeros_like(bh), np.zeros_like(by) # memory variables for Adagrad
smooth_loss = -np.log(1.0/vocab_size)*seq_length # loss at iteration 0

while True:
  # prepare inputs (we're sweeping from left to right in steps seq_length long)
  if p+seq_length+1 >= len(data) or n == 0: 
    hprev = np.zeros((hidden_size,1)) # reset RNN memory
    p = 0 # go from start of data
  inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
  targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]

  # sample from the model now and then
  if n % 100 == 0:
    sample_ix = sample(hprev, inputs[0], 200)
    txt = ''.join(ix_to_char[ix] for ix in sample_ix)
    print('----\n %s \n----' % (txt, ))

  # forward seq_length characters through the net and fetch gradient
  loss, dWxh, dWhh, dWhy, dbh, dby, hprev = lossFun(inputs, targets, hprev)
  smooth_loss = smooth_loss * 0.999 + loss * 0.001
  if n % 100 == 0: 
        print('iter %d, loss: %f' % (n, smooth_loss)) # print progress
  
  # perform parameter update with Adagrad
  for param, dparam, mem in zip([Wxh, Whh, Why, bh, by], 
                                [dWxh, dWhh, dWhy, dbh, dby], 
                                [mWxh, mWhh, mWhy, mbh, mby]):
    mem += dparam * dparam
    param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # adagrad update

  p += seq_length # move data pointer
  n += 1 # iteration counter 

----
 hennai super kings won ips won ips won ips won ips won ips won ips won ips won ips won ips won ips won ips won ips won ips won ips won ips won ips won ips won ips won ips won ips won ipsuper kings won 
----
iter 0, loss: 76.036948
----
 hennai super kings won ipngs won ipepspwon ipeenningsu peer iwngi skper kings won ipepsuper kings super ipngs won iper opngsrpeperi gs won ipngs won ipernnings srwon ipeper rings won ipnaipwon iper ai 
----
iter 100, loss: 69.687700
----
 hennai super kings won ipepsuper kings wonni kper kings won iper kings super iper kings wop ipers sepnaings seper ipnaings super aings super ipea ipkier kings won ipers won ipernningsusuper kings won  
----
iter 200, loss: 63.057131
----
 hennai super kings won ipepsrwopiipepnaingsuwrpep ipuper kings won ipngs won ipers won iper kipe  ingspwonki ipeper ki psuper iper ipnaings segsuwnnnii super kings won ipers won ipernnings sr epnnings 
----
iter 300, loss: 57.056225
----
 hennai super kings won ipers won iper ip

KeyboardInterrupt: 